In [1]:
import os
memory = '200g'
pyspark_submit_args = ' --driver-memory ' + memory  +' pyspark-shell' +" --num-executors 15 --executor-cores 8"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
import numpy as np
from collections import Counter
from HyperLogLog import Hyperloglog as hll
from HyperLogLog import Countsketch
from HyperLogLog import Data2sketch
import time
import pandas as pd
import pickle
import math
import random
import findspark
findspark.init()
from pyspark import SparkContext

sc=SparkContext(master="spark://Master:17077",appName="test")
#sc=SparkContext(appName="test")
sc._conf.set("spark.driver.maxResultSize", '50g')

In [2]:
def TransData(partition):
    level=1
    D2S=Data2sketch(width,level)
    D2S.ScanString([item.encode("utf-8") for item in partition])
    Item=[item.encode("utf-8") for item in partition]
    yield [[D2S.fiArray(i) for i in range(level)],D2S.NDVArray()]
def part2sketch(ss):
    level=1
    NDV=hll(width)
    hllarray=[hll(width) for i in range(level)]
    for item in ss:
        for ele in item:
            if ele[1]<=level:
                hllarray[ele[1]-1].add(str(ele[0]).encode("utf-8"))
            NDV.add(str(ele[0]).encode("utf-8"))
    data=[[h.Array() for h in hllarray],NDV.Array()]
    yield data
def GEE(NDV,f1,q):
    return f1*(math.sqrt(1/q)-1)+NDV
def ModifyChao(NDV,f1):
    return NDV+f1*(f1-1)/2/(NDV-f1+1)
def partComputef1(ss):
    sketch=pickle.loads(b.value)
    f1=0
    for s in ss:
        f1t=Computef1(sketch,s)
        f1+=f1t
    yield f1
def Computef1(SKList,F1ID):
    k_mechine=len(SKList)
    ZeroSketch=hll(width)
    IntersetID=[]
    for i in range(k_mechine):
        if i!=F1ID:
            ZeroSketch.update(SKList[i][1])
    MergeSketch=hll(width)
    MergeSketch.update(SKList[F1ID][0][0])
    f1=MergeSketch.estimate()
    MergeSketch.update(ZeroSketch.Array())
    f1pure=MergeSketch.estimate()-ZeroSketch.estimate()
    return f1pure

In [3]:
file1rdd=sc.textFile('/user/poi/poi-50-1kww/')
file1rdd.first()

'444087624'

In [4]:
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01

count n time: 29.21687078475952
n: 1000032019


In [5]:
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)

Collect Time: 39.35223722457886


In [ ]:
def Computef1(SKList,F1ID):
    k_mechine=len(SKList)
    ZeroSketch=hll(width)
    IntersetID=[]
    for i in range(k_mechine):
        if i!=F1ID:
            ZeroSketch.update(SKList[i][1])
    MergeSketch=hll(width)
    MergeSketch.update(SKList[F1ID][0][0])
    f1=MergeSketch.estimate()
    MergeSketch.update(ZeroSketch.Array())
    f1pure=MergeSketch.estimate()-ZeroSketch.estimate()
    return f1pure

In [ ]:
NDVSketch=[[]]
UniqueSketch=[]
times=1
for sk in SKList:
    temp=hll(width)
    temp.update(sk[1])
    UniqueSketch.append(temp)
    level=0
    temp=hll(width)
    temp.update(sk[0][0])
    NDVSketch[level].append(temp)
    level+=1
    m=times
    for level in range(1,len(NDVSketch)):
        if m==2**level*len(NDVSketch[level]):
            NDVSketch[level][-1].update(NDVSketch[level-1][-2].Array())
            NDVSketch[level][-1].update(NDVSketch[level-1][-1].Array())
        if m==2**level*len(NDVSketch[level])+1:
            temp=hll(width)
            NDVSketch[level].append(temp)
        if len(NDVSketch[-1])==3:
            NDVSketch.append([])
            temp=hll(width)
            NDVSketch[-1].append(temp)
            NDVSketch[-1][0].update(NDVSketch[-2][0].Array())
            NDVSketch[-1][0].update(NDVSketch[-2][1].Array())
            NDVSketch[-1].append([])
        times+=1
def query(NDVSketch,index,UniqueSketch):
    level=len(NDVSketch)-1
    result=hll(width)
    base=0
    esti=0
    if index<2**level:
        # left
        level-=1
        base=0
        while level>=0:
            if index<(base+1)*2**level:
                result.update(NDVSketch[level][base+1].Array())
                base=base*2
            else:
                result.update(NDVSketch[level][base].Array())
                base=(base+1)*2
            level-=1
        # right
        machine=len(NDVSketch[0])
        level=0
        while 2**level<=len(NDVSketch[0])-2**(len(NDVSketch)-1):
            #print(level,machine,ndv[level][machine-1])
            if machine%2==0 and machine//2!=1:
                machine=machine//2
            else:
                result.update(NDVSketch[level][machine-1].Array())
                machine=(machine-1)//2
            level+=1
        #result.append([-NDVSketch[0][index][0]])
    else:
        #left
        result.update(NDVSketch[-1][0].Array())
        #right
        machine=len(NDVSketch[0])
        level=0
        while 2**level<=len(NDVSketch[0])-2**(len(NDVSketch)-1):
            #print(level,machine,ndv[level][machine-1])
            if machine%2==0 and machine//2!=1:
                machine=machine//2
            else:
                if index>machine*2**level-2**level-1 and index<=machine*2**level-1:
                    tempindex=index
                    for newlevel in range(level):
                        result.update(ndv[newlevel][tempindex^1].Array())
                        tempindex=tempindex//2
                else:
                    result.update(ndv[level][machine-1].Array())
                machine=(machine-1)//2
            level+=1
        #result.append([-NDVSketch[0][index][0]])
    esti-=result.estimate()
    result.update(UniqueSketch[index].Array())
    esti+=result.estimate()
    return esti
f1=0
for index in range(len(SKList)):
    f1+=query(NDVSketch,index,UniqueSketch)
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()

In [6]:
#NDVhll=hll(width)
#for i in a:
#    NDVhll.update(i[1])
#NDV=NDVhll.estimate()
#Part_num=len(a)
#b=sc.broadcast(pickle.dumps(a))
#begin=time.time()
#Aimrdd=sc.parallelize([i for i in range(Part_num)])
#result=Aimrdd.mapPartitions(partComputef1).collect()
#f1=sum(result)

DGEE=GEE(NDV,f1,q)
end=time.time()
print("GEE Time:",end-begin)
DChao=ModifyChao(NDV,f1)
chend=time.time()
print("Chao Time",chend-begin)
resapfi = sc.parallelize(file1rdd.take(int(n*q))).map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)\
    .map(lambda d:(d[1],1)).reduceByKey(lambda e,f:e+f).collect()
sampleNDV=0
samplef1=0
for (key,value) in resapfi:
    sampleNDV+=value
    if key==1:
        samplef1=value
DShlosser=NDV+f1*(NDV-sampleNDV)/samplef1
shend=time.time()
print("Shlosser Time:",end-begin+shend-chend)
print("d:",NDV)
print("f1:",f1)
print("GEE:",DGEE)
print("Chao:",DChao)
print("Shlosser:",DShlosser)

GEE Time: 2.945502758026123
Chao Time 2.946061611175537
Shlosser Time: 21.31751537322998
d: 787267195.931828
f1: 605086523.0236659
GEE: 6233045903.144821
Chao: 1792120487.6820552
Shlosser: 48051914003.0267


In [7]:
file1rdd=sc.textFile('/user/poi/poi-50-5kww/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
Part_num=len(a)
b=sc.broadcast(pickle.dumps(a))
begin=time.time()
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DGEE=GEE(NDV,f1,q)
end=time.time()
print("GEE Time:",end-begin)
DChao=ModifyChao(NDV,f1)
chend=time.time()
print("Chao Time",chend-begin)
resapfi = sc.parallelize(file1rdd.take(int(n*q))).map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)\
    .map(lambda d:(d[1],1)).reduceByKey(lambda e,f:e+f).collect()
sampleNDV=0
samplef1=0
for (key,value) in resapfi:
    sampleNDV+=value
    if key==1:
        samplef1=value
DShlosser=NDV+f1*(NDV-sampleNDV)/samplef1
shend=time.time()
print("Shlosser Time:",end-begin+shend-chend)
print("d:",NDV)
print("f1:",f1)
print("GEE:",DGEE)
print("Chao:",DChao)
print("Shlosser:",DShlosser)

count n time: 79.08543157577515
n: 5000084437
Collect Time: 131.16143321990967
GEE Time: 11.32073426246643
Chao Time 11.321361780166626
Shlosser Time: 74.56890630722046
d: 3971129021.0032535
f1: 3104075342.728803
GEE: 31907807105.56248
Chao: 9527465358.96543
Shlosser: 248602678803.78427


In [8]:
file1rdd=sc.textFile('/user/poi/poi-50-1www/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
Part_num=len(a)
b=sc.broadcast(pickle.dumps(a))
begin=time.time()
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DGEE=GEE(NDV,f1,q)
end=time.time()
print("GEE Time:",end-begin)
DChao=ModifyChao(NDV,f1)
chend=time.time()
print("Chao Time",chend-begin)
resapfi = sc.parallelize(file1rdd.take(int(n*q))).map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)\
    .map(lambda d:(d[1],1)).reduceByKey(lambda e,f:e+f).collect()
sampleNDV=0
samplef1=0
for (key,value) in resapfi:
    sampleNDV+=value
    if key==1:
        samplef1=value
DShlosser=NDV+f1*(NDV-sampleNDV)/samplef1
shend=time.time()
print("Shlosser Time:",end-begin+shend-chend)
print("d:",NDV)
print("f1:",f1)
print("GEE:",DGEE)
print("Chao:",DChao)
print("Shlosser:",DShlosser)

count n time: 154.47623562812805
n: 9999969413
Collect Time: 237.5415985584259
GEE Time: 26.209994316101074
Chao Time 26.21020269393921
Shlosser Time: 134.90034532546997
d: 7925433568.293607
f1: 6156811768.681699
GEE: 63336739486.4289
Chao: 18641780934.27819
Shlosser: 492107899737.4306


In [9]:
file1rdd=sc.textFile('/user/zipf/zipf-2-1kww/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
Part_num=len(a)
b=sc.broadcast(pickle.dumps(a))
begin=time.time()
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DGEE=GEE(NDV,f1,q)
end=time.time()
print("GEE Time:",end-begin)
DChao=ModifyChao(NDV,f1)
chend=time.time()
print("Chao Time",chend-begin)
resapfi = sc.parallelize(file1rdd.take(int(n*q))).map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)\
    .map(lambda d:(d[1],1)).reduceByKey(lambda e,f:e+f).collect()
sampleNDV=0
samplef1=0
for (key,value) in resapfi:
    sampleNDV+=value
    if key==1:
        samplef1=value
DShlosser=NDV+f1*(NDV-sampleNDV)/samplef1
shend=time.time()
print("Shlosser Time:",end-begin+shend-chend)
print("d:",NDV)
print("f1:",f1)
print("GEE:",DGEE)
print("Chao:",DChao)
print("Shlosser:",DShlosser)

count n time: 25.738985776901245
n: 999590784
Collect Time: 41.590043783187866
GEE Time: 3.0444674491882324
Chao Time 3.0446937084198
Shlosser Time: 19.73644208908081
d: 982111665.846918
f1: 970160533.4956743
GEE: 9713556467.307987
Chao: 40359609687.17476
Shlosser: 95367582062.72543


In [10]:
file1rdd=sc.textFile('/user/zipf/zipf-2-5kww/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
Part_num=len(a)
b=sc.broadcast(pickle.dumps(a))
begin=time.time()
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DGEE=GEE(NDV,f1,q)
end=time.time()
print("GEE Time:",end-begin)
DChao=ModifyChao(NDV,f1)
chend=time.time()
print("Chao Time",chend-begin)
resapfi = sc.parallelize(file1rdd.take(int(n*q))).map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)\
    .map(lambda d:(d[1],1)).reduceByKey(lambda e,f:e+f).collect()
sampleNDV=0
samplef1=0
for (key,value) in resapfi:
    sampleNDV+=value
    if key==1:
        samplef1=value
DShlosser=NDV+f1*(NDV-sampleNDV)/samplef1
shend=time.time()
print("Shlosser Time:",end-begin+shend-chend)
print("d:",NDV)
print("f1:",f1)
print("GEE:",DGEE)
print("Chao:",DChao)
print("Shlosser:",DShlosser)

count n time: 78.82403826713562
n: 4998045354
Collect Time: 135.19188451766968
GEE Time: 16.203277826309204
Chao Time 16.20352554321289
Shlosser Time: 78.31809830665588
d: 4932968018.070068
f1: 4846064639.790714
GEE: 48547549776.18649
Chao: 140050512830.065
Shlosser: 478560136894.0397


In [11]:
file1rdd=sc.textFile('/user/zipf/zipf-2-1www/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
Part_num=len(a)
b=sc.broadcast(pickle.dumps(a))
begin=time.time()
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DGEE=GEE(NDV,f1,q)
end=time.time()
print("GEE Time:",end-begin)
DChao=ModifyChao(NDV,f1)
chend=time.time()
print("Chao Time",chend-begin)
resapfi = sc.parallelize(file1rdd.take(int(n*q))).map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)\
    .map(lambda d:(d[1],1)).reduceByKey(lambda e,f:e+f).collect()
sampleNDV=0
samplef1=0
for (key,value) in resapfi:
    sampleNDV+=value
    if key==1:
        samplef1=value
DShlosser=NDV+f1*(NDV-sampleNDV)/samplef1
shend=time.time()
print("Shlosser Time:",end-begin+shend-chend)
print("d:",NDV)
print("f1:",f1)
print("GEE:",DGEE)
print("Chao:",DChao)
print("Shlosser:",DShlosser)

count n time: 153.04342818260193
n: 9926629697
Collect Time: 238.03894329071045
GEE Time: 24.322214603424072
Chao Time 24.322423458099365
Shlosser Time: 132.2977213859558
d: 9819772637.430153
f1: 9730099133.778145
GEE: 97390664841.43344
Chao: 537706072558.57446
Shlosser: 962987972922.8815
